# three-body choregraphy

In this notebook we will present multiple example of three-body problem where the solution is remarquable and already done in this [gallery](http://three-body.ipb.ac.rs/).

We solve a restriction of $n$-body problem in 2 dimensions (we can remove $z$ direction without loss of generality). We note vector position $r_i=(x_i,y_i)$ of the mass $m_i$, $i=1,2,3$. The problem can be write as

$$
  \begin{aligned}
    \ddot{r}_1 &= -Gm_2\frac{r_1 - r_2}{|r_1-r_2|^3} - Gm_3\frac{r_1 - r_3}{|r_1-r_3|^3} \\
    \ddot{r}_2 &= -Gm_1\frac{r_2 - r_1}{|r_2-r_1|^3} - Gm_3\frac{r_2 - r_3}{|r_2-r_3|^3} \\
    \ddot{r}_3 &= -Gm_1\frac{r_3 - r_1}{|r_3-r_1|^3} - Gm_2\frac{r_3 - r_2}{|r_3-r_2|^3} \\
  \end{aligned}
$$

In the restriction we also set all mass to $1$: $m_i = 1$, $i=1,2,3$, and gravitational constant $G=1$ (the problem is nondimensioned).

Ponio is build to solve first order ODE, so we need to rewrite our problem with a change of variable:

$$
  \begin{aligned}
    \dot{y}_0 &= x_1 \\
    \dot{y}_1 &= y_1 \\
    \dot{y}_2 &= x_2 \\
    \dot{y}_3 &= y_2 \\
    \dot{y}_4 &= x_3 \\
    \dot{y}_5 &= y_3 \\
    \dot{y}_6 &= \dot{x}_1 \\
    \dot{y}_7 &= \dot{y}_1 \\
    \dot{y}_8 &= \dot{x}_2 \\
    \dot{y}_9 &= \dot{y}_2 \\
    \dot{y}_{10} &= \dot{x}_3 \\
    \dot{y}_{11} &= \dot{y}_3 \\
  \end{aligned}
$$

and the solve the problem done by

$$
  \begin{aligned}
    \dot{y}_0 &= \dot{y}_6 \\
    \dot{y}_1 &= \dot{y}_7 \\
    \dot{y}_2 &= \dot{y}_8 \\
    \dot{y}_3 &= \dot{y}_9 \\
    \dot{y}_4 &= \dot{y}_{10} \\
    \dot{y}_5 &= \dot{y}_{11} \\
    \dot{y}_6 &= - \frac{y_0 - y_2}{|r_1-r_2|^3} - \frac{y_0 - y_4}{|r_1-r_2|^3} \\
    \dot{y}_7 &= - \frac{y_1 - y_3}{|r_1-r_2|^3} - \frac{y_1 - y_5}{|r_1-r_2|^3} \\
    \dot{y}_8 &= - \frac{y_2 - y_0}{|r_1-r_2|^3} - \frac{y_2 - y_4}{|r_1-r_2|^3} \\
    \dot{y}_9 &= - \frac{y_3 - y_1}{|r_1-r_2|^3} - \frac{y_3 - y_5}{|r_1-r_2|^3} \\
    \dot{y}_{10} &= - \frac{y_4 - y_0}{|r_1-r_2|^3} - \frac{y_4 - y_2}{|r_1-r_2|^3} \\
    \dot{y}_{11} &= - \frac{y_5 - y_1}{|r_1-r_2|^3} - \frac{y_5 - y_3}{|r_1-r_2|^3} \\
  \end{aligned}
$$

with $r_1 = (y_0, y_1)$, $r_2 = (y_2, y_3)$ and $r_3 = (y_4, y_5)$.

In [ ]:
%system mkdir -p n_body_demo

In [ ]:
%%writefile n_body_demo/n_body.cpp

#include <iostream>
#include <numeric>
#include <valarray>
#include <numbers>
#include <cstdlib>
#include <filesystem>

#include <solver/problem.hpp>
#include <solver/solver.hpp>
#include <solver/observer.hpp>
#include <solver/butcher_methods.hpp>

struct nbody_model
{
    using vector_t = std::valarray<double>;
    using state_t  = std::valarray<double>;
    
    double G;
    double m1, m2, m3;
    std::size_t ncoord;
  
    nbody_model()
    : G(1.), m1(1.), m2(1.), m3(1.), ncoord(2)
    {}
    
    vector_t frac( vector_t const& u )
    {
        double norm = 1./std::pow(std::sqrt(u[0]*u[0] + u[1]*u[1]),3);
        return norm*u;
    }
    
    state_t operator () (double t , state_t const& y)
    {
        vector_t r1={y[0*ncoord+0], y[0*ncoord+1]},
                 r2={y[1*ncoord+0], y[1*ncoord+1]},
                 r3={y[2*ncoord+0], y[2*ncoord+1]};
        vector_t dr1 = - G*m2*frac(r1-r2) - G*m3*frac(r1-r3);
        vector_t dr2 = - G*m3*frac(r2-r3) - G*m1*frac(r2-r1);
        vector_t dr3 = - G*m1*frac(r3-r1) - G*m2*frac(r3-r2);
        
       return {
           y[3*ncoord+0],y[3*ncoord+1],
           y[4*ncoord+0],y[4*ncoord+1],
           y[5*ncoord+0],y[5*ncoord+1],
           dr1[0],dr1[1],
           dr2[0],dr2[1],
           dr3[0],dr3[1]
       };
    }
};

int main(int argc, char** argv)
{
    auto A = nbody_model();
    auto nbody_model_pb  = ode::make_simple_problem(A);
    
    std::filesystem::path output = "n_body/orbit.txt";
    double tf = 10.0;
    nbody_model::state_t  yini;
    
    if (argc > 1) {
        output = argv[1];
        
        if (argc == 5) {
            double p1 = std::atof(argv[2]);
            double p2 = std::atof(argv[3]);
            tf = std::atof(argv[4]);
            
            yini = {
                -1,0,
                1,0,
                0,0,
                p1,p2,
                p1,p2,
                -2*p1,-2*p2
            };
        } else if (argc == 15) {
            double x1 = std::atof(argv[2]), y1 = std::atof(argv[3]);
            double x2 = std::atof(argv[4]), y2 = std::atof(argv[5]);
            double x3 = std::atof(argv[6]), y3 = std::atof(argv[7]);
            
            double vx1 = std::atof(argv[ 8]), vy1 = std::atof(argv[ 9]);
            double vx2 = std::atof(argv[10]), vy2 = std::atof(argv[11]);
            double vx3 = std::atof(argv[12]), vy3 = std::atof(argv[13]);
            
            tf = std::atof(argv[14]);
            
            yini = {
                x1, y1,
                x2, y2,
                x3, y3,
                vx1, vy1,
                vx2, vy2,
                vx3, vy3
            };
        }
    }

    double dt = 1e-5;

    ode::solve(nbody_model_pb, ode::butcher::rk54_7m(1e-6), yini, {0.,tf}, dt, observer::file_observer(output) );
   
    return 0;
}

In [ ]:
%system $CXX -std=c++20 -I ../include -I ${CONDA_PREFIX}/include n_body_demo/n_body.cpp -o n_body_demo/n_body

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import subprocess

def launch(label, *args):
    tf = args[-1]
    opt = args[:-1]
    if len(opt) == 2:
        args = "{} {}".format(*opt)
    else:
        args = "{} {} {} {} {} {} {} {} {} {} {} {}".format(*opt)
        
    cmd = subprocess.Popen("./n_body_demo/n_body n_body_demo/{}.txt {} {}".format(label,args,tf).split())
    cmd.wait()
    
    data = np.loadtxt("n_body_demo/{}.txt".format(label))
    t = data[:,0]
    x1 = data[:,2*0+1]
    y1 = data[:,2*0+2]
    x2 = data[:,2*1+1]
    y2 = data[:,2*1+2]
    x3 = data[:,2*2+1]
    y3 = data[:,2*2+2]
    
    plt.plot(x1, y1, "-", linewidth=1)
    plt.plot(x2, y2, "-", linewidth=1)
    plt.plot(x3, y3, "-", linewidth=1)
    
    plt.show()

[Figure 8](http://three-body.ipb.ac.rs/sol.php?id=1)

In [ ]:
launch("figure8", 0.347111, 0.532728, 6.324449)

[Butterfly IV](http://three-body.ipb.ac.rs/sol.php?id=11)

In [ ]:
launch("butterflyIV", 0.350112, 0.079339, 79.475875)

In [ ]:
launch("butterflyIII", 0.405916, 0.230163, 13.865763)

In [ ]:
launch("dragonfly", 0.080584, 0.588836, 21.270975)

In [ ]:
launch("mothII", 0.439166, 0.452968, 28.670278)

In [ ]:
launch("BrouckeA1",
           -0.9892620043,0.0000000000,
           2.2096177241,0.0000000000,
           -1.2203557197,0.0000000000,
           0.0000000000,1.9169244185,
           0.0000000000,0.1910268738,
           0.0000000000,-2.1079512924
       , 6.283213)

In [ ]:
launch("ovals_flourishes",
           0.716248295713, 0.384288553041,
           0.086172594591, 1.342795868577,
           0.538777980808, 0.481049882656,
           1.245268230896, 2.444311951777,
           -0.675224323690,-0.962879613630,
           -0.570043907206,-1.481432338147
       , 8.094721)

In [ ]:
launch("ovals_catface_starship",
          0.536387073390, 0.054088605008,
          -0.252099126491, 0.694527327749,
          -0.275706601688, -0.335933589318,
          -0.569379585581, 1.255291102531,
          0.079644615252, -0.458625997341,
          0.489734970329, -0.796665105189
       , 5.026055)

In [ ]:
launch("skinny_pineapple",
        0.419698802831  , 1.190466261252,
        0.076399621771  , 0.296331688995,
        0.100310663856  , -0.729358656127,
        0.102294566003  ,  0.687248445943,
        0.148950262064  ,  0.240179781043,
        -0.251244828060 ,  -0.927428226977
       , 5.095054)

In [ ]:
launch("PT1",
        0.708322208567, 0.473311928207,
        0.167739458699, -0.057913961029,
        -0.506578687024, -0.306825234531,
        0.824266639920, 0.522197827478,
        -0.077017015655, -0.167288552679,
        -0.747249624265, -0.354909274800
       , 5.403011)

In [ ]:
launch("PT2",
        0.865355422048 , 0.629488893636,
        0.085036793559 , -0.013305780704,
        -0.090983494772, -0.892179296799,
        0.288687787607 , 0.171289709267,
        -0.220256752039, 0.090375753071,
        -0.068431035568, -0.261665462337
       , 5.427986)

In [ ]:
launch("butterflyI.2.A",
        0.306893,
        0.125507,
       6.234671)

In [ ]:
launch("butterflyI.2.B",
        0.392955,
        0.097579,
       7.003707)

In [ ]:
launch("butterflyI.5.A",
        0.411293,
        0.260755,
       20.749072)

In [ ]:
launch("butterflyI.9.A",
        0.402712,
        0.210016,
       34.711515)

In [ ]:
launch("butterflyI.11.A",
        0.415251,
        0.291346,
       47.925856)

In [ ]:
launch("butterflyI.12.A",
        0.408211,
        0.243685,
       48.486856)

In [ ]:
launch("butterflyI.14.A",
        0.415169,
        0.295341,
       61.323559)

In [ ]:
launch("butterflyI.16.A",
        0.404132,
        0.219164,
       62.446820)

In [ ]:
launch("butterflyI.17.A",
        0.397220,
        0.169198,
       62.627594)

In [ ]:
launch("butterflyI.19.A",
        0.407376,
        0.238843,
       76.221970)

In [ ]:
launch("butterflyI.20.A",
        0.401559,
        0.202266,
       76.401096)

In [ ]:
launch("butterflyI.21.A",
        0.396058,
        0.158601,
       76.592744)

In [ ]:
launch("butterflyI.23.A",
        0.404679,
        0.222598,
       90.181106)

In [ ]:
launch("butterflyI.25.A",
        0.395290,
        0.150852,
       90.562075)

In [ ]:
launch("dragonflyII.4.A",
        0.080584,
        0.588836,
       21.272338)

In [ ]:
launch("dragonflyII.6.A",
        0.186238,
        0.578714,
       33.641422)

In [ ]:
launch("dragonflyII.8.A",
        0.144812,
        0.542898,
       38.062055)

In [ ]:
launch("dragonflyII.14.A",
        0.108253,
        0.609812,
       82.135651)

In [ ]:
launch("dragonflyII.14.B",
        0.074732,
        0.567936,
       68.991156)

In [ ]:
launch("dragonflyII.15.A",
        0.049051,
        0.590194,
       79.152719)

In [ ]:
launch("dragonflyII.15.C",
        0.047547,
        0.564659,
       72.400492)

In [ ]:
launch("dragonflyII.15.D",
        0.179107,
        0.572603,
       81.599501)

In [ ]:
launch("dragonflyII.16.A",
        0.073903,
        0.619865,
       95.810473)

In [ ]:
launch("dragonflyII.17.A",
        0.061053,
        0.609177,
       96.873248)

In [ ]:
launch("dragonflyII.17.B",
        0.050367,
        0.570341,
       83.684255)

In [ ]:
launch("dragonflyII.17.C",
        0.179557,
        0.581300,
       95.474382)

In [ ]:
launch("yingyangIII.3.A.a",
        0.513938,
        0.304736,
       17.328810)

In [ ]:
launch("yingyangIII.3.A.b",
        0.282699,
        0.327209,
       10.963252)

In [ ]:
launch("yingyangIII.9.A.a",
        0.513150,
        0.289437,
       50.407145)

In [ ]:
launch("yingyangIII.12.A.a",
        0.416822,
        0.330333,
       55.789329)

In [ ]:
launch("yingyangIII.12.A.b",
        0.417343,
        0.313100,
       54.208001)

In [ ]:
launch("yingyangIII.13.A.a",
        0.416444,
        0.336397,
       63.406504)

In [ ]:
launch("yingyangIII.13.A.b",
        0.415819,
        0.306804,
       60.150751)

In [ ]:
launch("yingyangIII.15.A.a",
        0.414396,
        0.339223,
       70.492561)

In [ ]:
launch("yingyangIII.15.A.b",
        0.417701,
        0.303455,
       66.751098)

In [ ]:
launch("yingyangIII.15.B.a",
        0.513063,
        0.296863,
       85.129064)

In [ ]:
launch("yingyangIII.15.B.b",
        0.282036,
        0.325643,
       54.639089)

In [ ]:
launch("yingyangIII.15.C.a",
        0.516228,
        0.311409,
       88.440089)

In [ ]:
launch("yingyangIII.15.C.b",
        0.280396,
        0.329229,
       54.820022)

In [ ]:
launch("yingyangIII.16.A.a",
        0.427659,
        0.340300,
       80.146699)

In [ ]:
launch("yingyangIII.16.A.b",
        0.429098,
        0.299359,
       74.845173)

In [ ]:
launch("yingyangIII.18.A.a",
        0.413720,
        0.341698,
       84.855117)

In [ ]:
launch("yingyangIII.18.A.b",
        0.414643,
        0.301216,
       79.283684)

In [ ]:
launch("yingyangIII.19.A.b",
        0.418091,
        0.299900,
       86.360447)

In [ ]:
launch("yingyangIII.21.A.b",
        0.418259,
        0.299482,
       92.981286)

In [ ]:
launch("butterflyIVb.3.A",
        0.405916,
        0.230163,
       13.867124)

Et si on faisait une fonction pour récupérer directement tous les cas tests ? puis les lancer. À améliorer :

* ne faire que extraire les données des différents cas tests (nom, période et p1, p2)
* exporter ceci en json
* code C++ prend en entrer le fichier json, et sort un fichier text de ponio
* pytest pour comparer les résultats de ponio avec ref (générer par ponio avec un solveur et un pas de temps tout petit ? avec SciPy ? avec la figure de ref des exemples ?)
* lancer ceci depuis ailleurs que le réseau instable du train

In [ ]:
import urllib.request
import re

begin = "var text = "
end = "\n\n\n\n\n\n// Here you can put in the text you want to make it type"

for i in range(31):
    with urllib.request.urlopen('http://three-body.ipb.ac.rs/sIVb_sol.php?id={}'.format(i)) as response:
        html = response.read()
        
        idx = html.find(bytes(begin,"utf-8"))
        jdx = html.find(bytes(end,"utf-8"))
        
        data = html[idx+len(begin)+1:jdx-7].decode("utf-8")
        
        name = re.findall("NAME: ([^*]*) \\\\n DISCOVERED",data)[0]
        periode = float(re.findall("PERIOD: ([^*]*)\\\\n ENERGY",data)[0])
        p1 = float(re.findall("p1: ([^*]*)\\\\n p2",data)[0])
        p2 = float(re.findall("p2: ([^*]*)\\\\n",data)[0])
        
        print(f"butterfly{name}",p1,p2,periode)
        launch(f"butterfly{name}",p1,p2,periode)

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML
from scipy.interpolate import interp1d

def anime(label):
    data = np.loadtxt("n_body_demo/{}.txt".format(label))
    t = data[:,0]
    success_t = (t[1:] - t[:-1]) != 0
    
    t1 = t[:-1][success_t]
    x1 = data[:,2*0+1][:-1][success_t]
    y1 = data[:,2*0+2][:-1][success_t]
    x2 = data[:,2*1+1][:-1][success_t]
    y2 = data[:,2*1+2][:-1][success_t]
    x3 = data[:,2*2+1][:-1][success_t]
    y3 = data[:,2*2+2][:-1][success_t]
    
    #plt.plot(x1[:-1][success_t],y1[:-1][success_t],lw=1)
    #plt.plot(x2[:-1][success_t],y2[:-1][success_t],lw=1)
    #plt.plot(x3[:-1][success_t],y3[:-1][success_t],lw=1)
    
    fx1 = interp1d(t1,x1,kind='cubic')
    fy1 = interp1d(t1,y1,kind='cubic')
    
    fx2 = interp1d(t1,x2,kind='cubic')
    fy2 = interp1d(t1,y2,kind='cubic')
    
    fx3 = interp1d(t1,x3,kind='cubic')
    fy3 = interp1d(t1,y3,kind='cubic')
    
    
    time = np.linspace(t1[0],t1[-1],2000)
    X1, Y1 = (fx1(time), fy1(time))
    X2, Y2 = (fx2(time), fy2(time))
    X3, Y3 = (fx3(time), fy3(time))
    
    number_of_frames = np.size(time)
    
    def update_plot(n):
        r1.set_data(X1[max(0,n-100):n+1],Y1[max(0,n-100):n+1])
        r2.set_data(X2[max(0,n-100):n+1],Y2[max(0,n-100):n+1])
        r3.set_data(X3[max(0,n-100):n+1],Y3[max(0,n-100):n+1])

        m1.set_data([X1[n]],[Y1[n]])
        m2.set_data([X2[n]],[Y2[n]])
        m3.set_data([X3[n]],[Y3[n]])

        ax.set_title("n = {}".format(n))
        return r1, r2, r3, m1, m2, m3
    
    fig = plt.figure()
    ax = plt.axes(xlim=(-1.5, 1.5), ylim=(-1.6, 1.6))
    r1, = ax.plot([], [], '-', lw=1, color="#686de0")
    r2, = ax.plot([], [], '-', lw=1, color="#ffbe76")
    r3, = ax.plot([], [], '-', lw=1, color="#badc58")

    m1, = ax.plot([], [], 'o', color="#4834d4")
    m2, = ax.plot([], [], 'o', color="#f0932b")
    m3, = ax.plot([], [], 'o', color="#6ab04c")

    ani = animation.FuncAnimation(fig, update_plot, frames=number_of_frames, repeat=False, interval=20 )
    plt.close()
    return HTML(ani.to_html5_video())
    
#anime("figure8")
anime("dragonflyII.15.A")
#anime("yingyangIII.12.A.a")

In [ ]:
anime("figure8")